In [ ]:
# Import BeautifulSoup and splinter
from bs4 import BeautifulSoup as soup
from splinter import Browser

# Import other dependencies
import time
import pandas as pd
import numpy as np
from pathlib import Path
import requests
import json

# Import geoapify api key
geoapify_key = "f76b71d9aa0c4c4d8e5b01cff5e06c63"

In [ ]:
# Set up Splinter
browser = Browser('chrome')

### Scraping Top 25 for Hotels and Destinations

In [ ]:
# geoapify url endpoint
base_url = "https://api.geoapify.com/v1/geocode/search"

# List of urls and category used for scraping
url_list = []
category_list = []

url_list.append('https://www.tripadvisor.com/TravelersChoice-Hotels')
category_list.append('hotels')

url_list.append("https://www.tripadvisor.com/TravelersChoice-Destinations")
category_list.append("destinations")


In [ ]:
# loop thru url list that we need to scrape and get data (done only if csv of data is not present)

for index, item in enumerate(url_list):
    
    search_file = "static/data/" + category_list[index] + ".csv"
    csv_file = Path(search_file)

    if csv_file.exists():
        print(f"File {csv_file} already extracted. Skipping scrapping")
    else:
                
        # Set up Splinter
        browser = Browser('chrome')

        # Visit the website - sleep provided to avoid continuous calls   
        browser.visit(item)
        time.sleep(30)       
        # Optional delay for loading the page
        browser.is_element_present_by_css('div.list_text', wait_time=1)

        # Scrape the website html
        html = browser.html

        # Create a BeautifulSoup object from the scraped HTML
        data = soup(html, 'html.parser')

        # Get Rank and name
        names = data.find_all(class_="mainName extra")
        namelist = []
        ranklist = []
        parts = []
        for i in names:
            href=i.find("a")    
            text_data=href.text   
            parts = text_data.split('.')    
            ranklist.append(parts[0])
            namelist.append(parts[1].strip())
    
        # Get City and Country (single string)
        cities = data.find_all(class_="smaller")
        citylist = []
        lat = []
        lon = []
        for i in cities:
            href=i.find("a")
            city_loc = href.text    
            citylist.append(city_loc)
            
            # call geoapify to get lat/lon for location
            params = {
                "text": city_loc,
                "apiKey": geoapify_key }
            # Run request
            response = requests.get(base_url, params=params).json()
            
            # Extract lat/lon
            latitude = response["features"][0]["properties"]["lat"]
            longitude = response["features"][0]["properties"]["lon"]

            # append to list
            lat.append(latitude)
            lon.append(longitude)
    

        # Get image urls
        images = data.find_all(class_="sizedThumb_container")
        iurls = []
        for i in images:
            href=i.find("img")
            iurls.append(href["src"])

        # Get description of restaurant/a customer review 
        desclist = []
        quot_tags = data.find_all(class_="quot")
        # removing classes with "quot quot2" used for second quote as we need just 1 quote per target
        quot_tags = [tag for tag in quot_tags if 'quot2' not in ''.join(tag['class'])]
        for texts in quot_tags:
            x = texts.find("i").next_sibling.strip()        
            desclist.append(x)

        # Get url to go retrieve rating and reviews - this will be used later to get details
        url_ary = []
        for lnk in data.find_all(class_="firstone"):
            href=lnk.find("a")    
            url_ary.append(href['href'])

        # Close browser
        browser.quit() 

        # Create array with category and count of rank array
        cat_list = np.repeat([category_list[index]], len(ranklist))

        # Load arrays as columns of dataframe
        df = pd.DataFrame({"category": cat_list, "rank": ranklist, "name": namelist, "location": citylist, 
                       "imageurl": iurls, "description": desclist, "latitude": lat, "longitude": lon, "ratingurl": url_ary})
        
        # save dataframe as csv 
        filename = "static/data/" + category_list[index] + ".csv"
        df.to_csv(filename, encoding="utf-8", index=False, header=True)
        

    # sleep for a minute before calling next url (providing break to avoid continuous pings to website)
    time.sleep(10)

# Final checks
browser.quit()

### Scraping Reviews for Hotels Category

In [ ]:
rates = []
rvcount = []
exclnt = []
vgood = []
averg = []
poor = []
trrbl = []
rank_key = []

# Get rating and misc info for all hotels
prefix = "https://www.tripadvisor.com"

hotels_df = pd.read_csv("static/data/hotels.csv")
# loop thru hotels and scrape rating information and store as csv
for index, row in hotels_df.iterrows():
    rank_key.append(row['rank'])
    addlink = row['ratingurl']
    url = prefix + addlink

    # setup Splinter
    browser = Browser('chrome')

    # Visit the website - sleep provided to avoid continuous calls   
    browser.visit(url)
    # Scrape the website html
    html = browser.html

    # Create a BeautifulSoup object from the scraped HTML
    raters = soup(html, 'html.parser')
    browser.quit()
    time.sleep(15)

    # extract data and populate arrays
    # get overall rating (has decimals)
    rating_text = raters.find(class_="biGQs _P fiohW hzzSG uuBRH").text
    rating = float(rating_text)
    rates.append(rating)

    # get review count
    rcount_text = raters.find(class_="yyzcQ").text
    # remove all ',' separators and cast as int
    rcount_text = rcount_text.replace(',', '')
    rvcount.append(int(rcount_text))  

    # get the different types of ratings and their values
    v1 = raters.find_all(class_="IMmqe")
    for i in v1:
        rv = i.find(class_="biGQs _P pZUbB osNWb").text
        rv = rv.replace(',','') 
        rt = i.find(class_="biGQs _P pZUbB hmDzD").text

        if rt == "Excellent":
            exclnt.append(rv)
        elif rt == 'Very good':
            vgood.append(rv)
        elif rt == 'Average':
            averg.append(rv)
        elif rt == 'Poor':
            poor.append(rv)
        elif rt == 'Terrible':
            trrbl.append(rv)


In [ ]:
# add to dataframe and save as csv
catlist = np.repeat(["hotels"], len(rank_key))
df1 = pd.DataFrame({"category": catlist, "rank": rank_key, "rate": rates, "total_reviews": rvcount, "excellent": exclnt, 
                    "very_good": vgood, "average": averg, "poor": poor, "terrible": trrbl})
# save dataframe as csv 
filename = "static/data/hotelsreviews.csv"
df1.to_csv(filename, encoding="utf-8", index=False, header=True)